# Model 1: meta-llama/Meta-Llama-3-8B-Instruct

In [1]:
!pip install -U accelerate transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.5 MB/s eta 0:00:00


In [2]:
!pip install -i https://pypi.org/simple/ bitsandbytes --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00


In [3]:
!pip install accelerate --q

In [4]:
!pip install -U datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [1]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig

In [3]:
q_config = BitsAndBytesConfig (
      load_in_4bit = True,
      bnb_4bit_quant_type="nf4",
  )

In [4]:
def create_model_tokenizer(name, q_config):
  tokenizer = AutoTokenizer.from_pretrained(name)
  model = AutoModelForCausalLM.from_pretrained (
      name,
      quantization_config = q_config,
      device_map="cuda",
      torch_dtype="auto",
      trust_remote_code = True
  )
  return model, tokenizer

In [5]:
# Load model directly
model1, tokenizer1 = create_model_tokenizer("NousResearch/Meta-Llama-3-8B-Instruct", q_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Zero-shot Learning

In [6]:
generation_pipeline = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
    framework="pt",
)

In [7]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [8]:
question1 = "What is the capital of France?"
question2 = "Which city is the capital of France?"
# question1 = "How old are you?"
# question2 = "What is your age?"
prompt1 = f"Question 1: {question1}\nQuestion 2: {question2}\nAre these questions similar? Just say 'yes' or 'no'"

In [9]:
messages1 = [
    {"role": "user", "content": prompt1},
]

In [10]:
pipe1 = pipeline(
    "text-generation",
    model=model1,
    tokenizer=tokenizer1,
)

In [11]:
output1 = pipe1(messages1, **generation_args)
print(output1[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Yes


## One-shot Learning

In [20]:
from datasets import load_dataset
import random

In [17]:
dataset = load_dataset("persiannlp/parsinlu_entailment")
# for x in dataset['train']:
#   print(x)

In [18]:
dataset["train"][0]

{'sent1': 'زنان به قدری بخش بزرگی از نیروی کار را تشکیل می دهند که به سختی می توان باور داشت که اگر این امر در مورد زنان  صادق نباشد ، این امر می تواند صادق باشد.',
 'sent2': 'مردان بخش عظیمی از نیروی کار هستند بنابراین تنها افراد مهم هستند.',
 'category': 'translation-train',
 'label': 'c'}

In [21]:
random.choice(dataset["train"])

{'sent1': 'به نظر می\u200cرسد یکی از بزرگ\u200cترین مشکلات کشور و به\u200cطور خاص صنعت نفت، نداشتن نگاه کلی و جزئی\u200cنگری در امور است.',
 'sent2': 'ما برای اداره ی کشور خود برنامه ریزی صحیحی نداریم.',
 'category': 'natural-miras',
 'label': 'e'}

In [22]:
random.choice(dataset["test"])

{'sent1': 'پس از تحصیلات مقدماتی، در زمانی که پدرش، سفیر کبیر ایران در دربار عثمانی بود، علی\u200cرغم علاقه پدر و مادرش به تحصیل وی در فرانسه، در سال ۱۲۹۱ به دبیرستان نظام در استانبول ترکیه رفت، ولی پس از دو سال، به علت شروع جنگ به ایران آمد و با درجه افسری، به ارتش پیوست و پس از این ایام بود که به دانشکده افسری سویس اعزام شد و پس از آن در سال ۱۳۰۲ جزو محصلین اعزامی به فرانسه رفت و دوره تکمیلی\u200cدانشکده سواره نظام را طی کرد و به ایران بازگشت.',
 'sent2': 'پدر و مادرش علاقه داشتند وی در ترکیه تحصیل کند.',
 'category': 'natural-wiki',
 'label': 'c'}

In [28]:
train = random.choice(dataset["train"])
test = random.choice(dataset["test"])
# question1 = "How old are you?"
# question2 = "What is your age?"
prompt2 = f"Sentence 1: {train['sent1']}\nSentence 2: {train['sent2']}\nThe label for these 2 questions is:{train['label']}\nNow tell me this: Are these questions similar? Just say 'yes' or 'no'\n Sentence 3: {test['sent1']}\n Sentence 4: {test['sent2']}"

In [29]:
messages2 = [
    {"role": "user", "content": prompt2},
]

In [31]:
output2 = pipe1(messages2, **generation_args)
print(output2[0]['generated_text'])
print(test['label'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The label for these 2 questions is: "نقدی" (Economic)

And, yes, these questions are similar.
n


## Five-shot Learning

In [ ]:
# Initialization step
    viterbi[:, 0] = start_prob * emit_prob[:, obs_index[observations[0]]]

    # Recursion step
    for t in range(1, T):
        for s in range(N):
            viterbi[s, t] = np.max(viterbi[:, t - 1] * trans_prob[:, s]) * emit_prob[s, obs_index[observations[t]]]
            backpointer[s, t] = np.argmax(viterbi[:, t - 1] * trans_prob[:, s])

    # Termination step
    bestpathprob = np.max(viterbi[:, T - 1])
    bestpathptr = np.argmax(viterbi[:, T - 1])
    bestpath = [bestpathptr]

    # Path backtracking
    for t in range(T - 1, 0, -1):
        bestpathptr = backpointer[bestpathptr, t]
        bestpath.insert(0, bestpathptr)

    # Convert state indices back to state names
    bestpath = [states[state] for state in bestpath]